In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pickle
import os
from common_imports import *
from companion_script import *
%load_ext autoreload
case_id = "DMB006"
setup = pickle.load(file=open(os.path.join('patients',case_id,case_id+'_backup4_DISCoVER.p'),'rb'))

In [156]:
from rpy2.robjects import numpy2ri
numpy2ri.activate()
#     from discover import discover_from_expression, plot_discover_from_expression
from discover_temp import discover_from_expression, plot_discover_from_expression
from drug_suggestion.expression.controls import load_control_exp
patient_exp = pd.read_csv(setup.patient_gexp_file, index_col=0)
control_exp = load_control_exp(setup.expression_control)
log("About to perform DiSCoVER.")
discover_results = discover_from_expression(exp=patient_exp, 
                                            control_exp=control_exp, 
                                            verbose=True,extra_outputs=True)
setup.raw_discover_results = discover_results
#     pickle.dump(setup, file=open(setup.out_dir+'_backup4-0_DISCoVER.p','wb'))
#     print(discover_results)
# move some files created by DiSCoVER
#     for cl_name in ['ccle','ctrp','gdsc']:
#         os.rename(f"COSMIC_cell_lines_IDs_and_types_{cl_name}.csv", os.path.join(setup.out_dir, f"COSMIC_cell_lines_IDs_and_types_{cl_name}.csv"))

log("DiSCoVER done!")
numpy2ri.deactivate()
log("Moving files contaiend enrichment of cell lines")
for current_file in ['cell_lines_IDs_and_types_ccle.csv','cell_lines_IDs_and_types_COSMIC_IDS_gdsc.csv','cell_lines_IDs_and_types_ctrp.csv']:
    os.rename(current_file,os.path.join(setup.discover_out_dir,current_file))
    log(f'Moved {current_file} to {os.path.join(setup.discover_out_dir,current_file)}')
log("Ranking cell lines by enrichment and saving those")
ranked_diseases_from_enrichment = rank_cell_lines(setup)
setup.ranked_diseases_from_enrichment = ranked_diseases_from_enrichment

==> About to perform DiSCoVER.
Modifying it on 2018-11-26
Loading CTRP
Projecting disease gene sets
I'ts ssgsea...
ssGSEA results for ctrp
            DMB006
NCIH524   0.345268
NCIH1184  0.323423
DMS273    0.311811
HCC33     0.293539
D283MED   0.259917
Matching to drug response profiles
Writing file: cell_lines_IDs_and_types_ctrp.csv
==> DiSCoVER done!
==> Moving files contaiend enrichment of cell lines
==> Moved cell_lines_IDs_and_types_ccle.csv to /pdx-hts/Notebooks/patients/DMB006/drug_suggestions/discover/cerebellar_stem/cell_lines_IDs_and_types_ccle.csv
==> Moved cell_lines_IDs_and_types_COSMIC_IDS_gdsc.csv to /pdx-hts/Notebooks/patients/DMB006/drug_suggestions/discover/cerebellar_stem/cell_lines_IDs_and_types_COSMIC_IDS_gdsc.csv
==> Moved cell_lines_IDs_and_types_ctrp.csv to /pdx-hts/Notebooks/patients/DMB006/drug_suggestions/discover/cerebellar_stem/cell_lines_IDs_and_types_ctrp.csv
==> Ranking cell lines by enrichment and saving those
CCLE is missing 3 cell lines (out of 475)
C

In [157]:
%autoreload 1
from discover_temp import discover_from_expression
numpy2ri.activate()
discover_results = discover_from_expression(exp=patient_exp, 
                                            control_exp=control_exp, 
                                            verbose=True,
                                            extra_outputs=True)
numpy2ri.deactivate()

Modifying it on 2018-11-26
Loading CTRP
Projecting disease gene sets
I'ts ssgsea...
ssGSEA results for ctrp
            DMB006
NCIH524   0.345268
NCIH1184  0.323423
DMS273    0.311811
HCC33     0.293539
D283MED   0.259917
Matching to drug response profiles
Writing file: cell_lines_IDs_and_types_ctrp.csv


In [154]:
def standarize_string(what):
    ascii_8bit = re.sub(r'[^ -~]', '', what).lower() # keeping only 8-bit ascii, making lowercase (http://www.catonmat.net/blog/my-favorite-regex/)
    return re.sub(r'[ _-]', '', ascii_8bit) # remove space, underscore, and dash (" _-")


# the name of the enrichment is the same as the patient ID which is stored in setup.case_id
def add_rank(df,by,name):
    df = df.sort_values(ascending=False, by=setup.case_id)
    df[name] = df[by].rank(ascending=False)
    return df


# Take each disease and assign it a rank
def rank_diseases(df):
    rank_name = df.columns[-1]
    new_df = pd.DataFrame()
    to_add = pd.Series()
    new_ix = 0
    for ix,row in df.loc[:,['Disease',rank_name]].iterrows():
        for disease in row['Disease'].split('__&&__'):
            to_add['Disease'] = disease
            to_add[rank_name] = row[rank_name]
            to_add.name = new_ix
            new_df = new_df.append(to_add)
#             print(to_add)
            new_ix += 1
    return new_df


def average_disease_rank(df,rank_name):
    new_df = pd.DataFrame()
    for disease in np.unique(df['Disease']):
        new_df.loc[disease.lower(),'mean_rank'] = df[df['Disease']==disease].loc[:,rank_name].median()
        new_df.loc[disease.lower(),'weight'] = len(df[df['Disease']==disease].loc[:,rank_name])
    return new_df.sort_values(by='mean_rank')

In [158]:
def rank_cell_lines(setup):
    # first load some dictionaries
    cid2dic = pickle.load(file=open('discover_temp/cellosaurus/cellosaurus_cosmic_id_dic.p','rb'))
    name2dic = pickle.load(file=open('discover_temp/cellosaurus/cellosaurus_name_dic.p','rb'))
    
    # CCLE
    ccle = pd.read_csv(os.path.join(setup.discover_out_dir,"cell_lines_IDs_and_types_ccle.csv"),index_col=0)
    missing = 0
    for ix, row in ccle.iterrows():
        name = standarize_string(ix)
        try: 
            dic = name2dic[name]
        except KeyError:
            missing += 1
            dic = {}
            dic['Names'] = ix
            dic['Disease'] = 'N/A'
            dic['CellosaurusID'] = 'N/A'       
        ccle.loc[ix,'Names'] = dic['Names']
        ccle.loc[ix,'CellosaurusID'] = dic['CellosaurusID']
        ccle.at[ix,'Disease'] = dic['Disease']
    if missing >0:
        log(f'CCLE is missing {missing} cell lines (out of {len(ccle)})')
    ccle.to_excel(os.path.join(setup.discover_out_dir,"processed_cell_lines_info_ccle.xlsx"))
    # ccle.head()

    #CTRP
    ctrp = pd.read_csv(os.path.join(setup.discover_out_dir,"cell_lines_IDs_and_types_ctrp.csv"),index_col=0)
    missing = 0
    for ix, row in ctrp.iterrows():
        name = standarize_string(ix)
        try: 
            dic = name2dic[name]
        except KeyError:
            missing += 1
            dic = {}
            dic['Names'] = ix
            dic['Disease'] = 'N/A'
            dic['CellosaurusID'] = 'N/A'       
        ctrp.loc[ix,'Names'] = dic['Names']
        ctrp.loc[ix,'CellosaurusID'] = dic['CellosaurusID']
        ctrp.at[ix,'Disease'] = dic['Disease']
    if missing >0:
        log(f'CTRP is missing {missing} cell lines (out of {len(ctrp)})')
    ctrp.to_excel(os.path.join(setup.discover_out_dir,"processed_cell_lines_info_ctrp.xlsx"))
    # ctrp.head()

    # GDSC
    gdsc = pd.read_csv(os.path.join(setup.discover_out_dir,"cell_lines_IDs_and_types_COSMIC_IDS_gdsc.csv"),index_col=0)
    missing = 0
    for ix, row in gdsc.iterrows():
        cid = str(int(row['COSMIC ID']))
        try:
            dic = cid2dic[cid]
            gdsc.loc[ix,'Names'] = dic['Names']
            gdsc.at[ix,'Disease'] = dic['Disease']
            gdsc.loc[ix,'CellosaurusID'] = dic['CellosaurusID']
        except KeyError:
            missing+=1
    if missing>0:
        log(f"GDSC is missing {missing} cell lines (out of {len(gdsc)}). That's a bit troubling")
    gdsc.to_excel(os.path.join(setup.discover_out_dir,"processed_cell_lines_info_gdsc.xlsx"))
    # gdsc.head()
        
    # add ranks
    ccle = add_rank(df=ccle,by=setup.case_id,name='CCLE_rank')
    ctrp = add_rank(df=ctrp,by=setup.case_id,name='CTRP_rank')
    gdsc = add_rank(df=gdsc,by=setup.case_id,name='GDSC_rank')
    
    # Unique disease names, ranked
    ccle_rank = rank_diseases(df=ccle)
    ctrp_rank = rank_diseases(df=ctrp)
    gdsc_rank = rank_diseases(df=gdsc)
    
    #Merge the three databases
    merged = ccle_rank.rename({"CCLE_rank": "rank"}, axis='columns').append(ctrp_rank.rename({"CTRP_rank": "rank"}, axis='columns')).append(gdsc_rank.rename({"GDSC_rank": "rank"}, axis='columns'))
    merged['Disease'] = merged.apply(lambda x: x['Disease'].lower(), axis=1)
    
    # Only the merged rank will be printed for now
    # ccle_average_rank = average_disease_rank(df=ccle_rank,rank_name='CCLE_rank')
    # ctrp_average_rank = average_disease_rank(df=ctrp_rank,rank_name='CTRP_rank')
    # gdsc_average_rank = average_disease_rank(df=gdsc_rank,rank_name='GDSC_rank')
    merged_rank = average_disease_rank(df=merged,rank_name='rank')
    merged_rank.to_csv(os.path.join(setup.discover_out_dir,"cell_lines_rank.csv"))
    return merged_rank

CCLE is missing 3 cell lines (out of 475)
CTRP is missing 5 cell lines (out of 633)


In [150]:
print(len(ccle),len(ctrp),len(gdsc))

475 633 985


In [149]:
# ccle.head()

,DMB006,Names,CellosaurusID,Disease,CCLE_rank
NCIH1184,0.323423,NCI-H1184__&&__H1184__&&__H-1184__&&__NCIH1184,CVCL_1458,Small cell lung carcinoma__&&__Lymph node,1.0
NCIH1092,0.302924,NCI-H1092__&&__H1092__&&__H-1092,CVCL_1454,Small cell lung carcinoma__&&__Bone marrow,2.0
NCIH1155,0.255616,NCI-H1155__&&__H1155__&&__H-1155__&&__NCIH1155...,CVCL_1456,Large cell lung carcinoma__&&__Lymph node,3.0
NCIH1694,0.237658,NCI-H1694__&&__H1694__&&__H-1694,CVCL_1489,Small cell lung carcinoma__&&__Ascites,4.0
NCIH1341,0.213815,NCI-H1341__&&__H1341__&&__H-1341,CVCL_1463,Small cell lung carcinoma__&&__Cervix,5.0


In [59]:
ccle_rank.head()

,CCLE_rank,Disease
0,1.0,Small cell lung carcinoma
1,1.0,Lymph node
2,2.0,Small cell lung carcinoma
3,2.0,Bone marrow
4,3.0,Large cell lung carcinoma


In [145]:
# df.rename({"CCLE_rank": "rank"}, axis='columns')

# merged.head()

,Disease,rank
0,small cell lung carcinoma,1.0
1,lymph node,1.0
2,small cell lung carcinoma,2.0
3,bone marrow,2.0
4,large cell lung carcinoma,3.0


In [151]:
display(ccle_average_rank.head())
display(ctrp_average_rank.head())
display(gdsc_average_rank.head())
display(merged_rank.head())

,mean_rank,weight
cervix,5.0,1.0
brain,12.0,3.0
adrenal gland neuroblastoma,15.0,1.0
pancreatic somatostatinoma,20.0,1.0
prostate carcinoma,21.0,2.0


,mean_rank,weight
cervix,10.0,1.0
small cell lung carcinoma,15.0,19.0
adrenal gland neuroblastoma,20.0,2.0
brain,23.0,5.0
vertebra,30.0,1.0


,mean_rank,weight
small cell lung carcinoma,33.5,58.0
lung carcinoma,35.0,1.0
thyroid gland squamous cell carcinoma,43.0,1.0
adrenal gland,44.5,2.0
supraclavicular lymph node,59.5,2.0


,mean_rank,weight
cervix,10.0,3.0
adrenal gland neuroblastoma,26.0,5.0
small cell lung carcinoma,28.5,88.0
lung carcinoma,35.0,1.0
pancreatic somatostatinoma,35.0,3.0


In [131]:
gdsc_average_rank.loc['cervix',:]

mean_rank    67.0
weight        1.0
Name: cervix, dtype: float64

mean_rank    74.0
weight        8.0
Name: medulloblastoma, dtype: float64

In [ ]:
# def run_discover(setup):
    
    
    
    
    
    log('Saving results to file.')
    # display(discover_results)
    discover_results.T.sort_values(by=setup.case_id, ascending=False).to_csv(setup.full_discover_results_file)
    log("Saving done!")
    log("NOT Restricting to clinically relevant drugs.")
    #Not all drugs in CCLE, CTRP, and GDSC are realistic candidates for treatment. We compiled a list of medications that are FDA-approved or in late-stage clinical trials, and Dr. Wechsler-Reya curated it to include only those that are relevant for treating brain tumors. Here we limit the results to these drugs and add Dr. Wechsler-Reya's mechanism-of-action annotations. To enable comparison of drug lists, drugs from the different sources have been mapped to PubChem compound IDs (CIDs) using [PubChemPy](http://pubchempy.readthedocs.io/en/latest/).
    from drug_suggestion.drug_annotation import subset_to_reasonable_drugs
    from drug_suggestion.expression.discover import load_discover_drug_to_cids
    disco2cid = load_discover_drug_to_cids()
    reasonable_results = subset_to_reasonable_drugs(discover_results, 
                                                disco2cid, 
                                                out_prefix='discover.{}'.format(setup.expression_control), 
                                                out_dir=setup.discover_out_dir)
    # This will override the file setup.rdrugs_discover_file
    all_drugs = format_drugs(discover_results, 
                                disco2cid, 
                                out_prefix='discover.{}'.format(setup.expression_control), 
                                out_dir=setup.discover_out_dir)
    log('Done restricting to clinically relevant drugs!')
    
##     log('making a discover illustrative method')
##     from drug_suggestion.expression.discover import plot_discover_from_expression
##     plot_discover_from_expression(case_id, 
##                                   discover_results, 
##                                   exp=patient_exp,
##                                   control_exp=control_exp,
##                                   cl='ctrp',
##                                   out_file=discover_heatmap_file)
##     make_discover_workflow_slide(mtb_ppt_file, discover_heatmap_file)
    log('Making the DiSCoVER powerpoint.')
    rdrugs_discover = pd.read_csv(setup.rdrugs_discover_file, index_col=None)

    
    # Using all of the drugs
    log('Using all of the drugs')
    df = split_discover_dataframe(df=rdrugs_discover)
    df = rank_drugs_discover(df)
#     df.head()
    make_exp_drug_ranking_results_slide(setup.mtb_ppt_file, df.head(20), setup.expression_control, method='DiSCoVER')
    log('Done making the DiSCoVER powerpoint slide!')
    log('Savig the variables to a file.')
    setup.discover_results = discover_results
    setup.disco2cid = disco2cid
    setup.control_exp = control_exp
    setup.reasonable_results = reasonable_results
    setup.df = df
    log('Saving the formatted results of DiSCoVER to a csv')
    df.to_csv(os.path.join(setup.out_dir, setup.case_id+'_formated_DISCoVER_results.csv'))
#     pickle.dump(setup, file=open(setup.out_dir+'_backup4_DISCoVER.p','wb'))
    pickle.dump(setup, file=open(os.path.join(setup.out_dir, setup.case_id+'_backup4_DISCoVER.p'),'wb'))
#     pickle.dump(setup, file=open(setup.case_id+'_DISCoVER.p','wb'))
    log('Done savig the variables to a file!')
    
    log('Done with all the taks in this cell. Move along.')
    return setup